In [ ]:
#set up keys
CONSUMER_KEY = '###'
CONSUMER_SECRET = '###'
ACCESS_TOKEN = '###'
ACCESS_TOKEN_SECRET = '###'
BEARER_TOKEN ='###'


In [ ]:
pip install xlsxwriter 

     |████████████████████████████████| 153kB 8.1MB/s 


In [ ]:
import tweepy
import time
import pandas as pd
import re
import numpy as np
import xlsxwriter

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)

auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
#load dataset of users
df = pd.read_excel('/WindWatchOrg_snowball_user_sample.xlsx')

In [ ]:
#filter on those who are followed by windwatchorg
windwatch = df[df.followed_by == 'windwatchorg']

In [ ]:
#filter on users who have "wind" in their bios or username, make to list
windwatch = windwatch[(windwatch.description.str.contains('(?i)Wind') == True) | 
                     (windwatch.username.str.contains('(?i)Wind') == True)]

usernames = windwatch.username.unique()

In [ ]:
new = []
while True:
    #try code, expect error to occur
    try: 
      #loop through list of usernames 
      for i in range(len(usernames)):
        print('Currently scraping:', usernames[i], i)
        #tweepy cursor object to get full user_timeline
        for tweet in tweepy.Cursor(api.user_timeline, id = usernames[i], tweet_mode = 'extended').items():

          #get username
          username = tweet._json['user']['screen_name']

          #get user id
          user_id = tweet._json['user']['id']

          #get tweet
          tweets = tweet._json['full_text']

          #get time of tweet
          timestamp = tweet._json['created_at']
          #get tweet id
          tweet_id = tweet._json['id']
          print(tweet_id)

          #dict of info, which gets stored in global list 
          data = {'username': username, 'user_id': user_id, 'tweet': tweets, 'timestamp': timestamp, 'tweet_id':tweet_id}                
          new.append(data)
    
    #pause for 5 sec before resuming after getting error(don't even think it's a rate limit thing, but just in case if it is)
    except BaseException as e:
        print('Failed on_status,',str(e)) # print the error code obtained from twitter
        print('Failed on account', usernames[i])
        time.sleep(5) # provide a time before resuming the code when an error arises
        print('Trying again')
    continue

In [ ]:
tweet_df = pd.DataFrame(new)

In [ ]:
#drop any hypothetical duplicate tweets
tweet_df.drop_duplicates()

In [ ]:
#write to csv
tweet_df.to_csv('resistance_tweets.csv')